<a href="https://colab.research.google.com/github/LipeLiposo/Chatbot_para_Agendamento_e_Atendimentos/blob/main/Imers%C3%A3o_IA_Alura_%2B_Google_Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importações e configs iniciais
!pip install -u -q google-generativeai
import numpy as np
import pandas as pd
import google.generativeai as genai
from google.colab import userdata

api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
#Definição e Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Titulo": "Tabela de preços",
    "Conteudo": "CORTE MASCULINO->R$35 / CORTE INFANTIL MASCULINO->R$35 / CORTE FEMININO->R$80 / CORTE INFANTIL FEMINO->R$50 / SOBRANCELHA->R$40 / SOBRANCELHA + HENNA->R$45 / APLICAÇÃO DE HENNA->R$18 / BARBA->R$30 / BUÇO->R$18 / DEPILAÇÃO FACIAL COM LINHA->R$60"}
DOCUMENT2 = {
    "Titulo": "Horário de Funcionamento",
    "Conteudo": "O horário de atendimento do salão de beleza é das 09:00 às 12:00 e das 14:00 às 22:00"
}
DOCUMENT3 = {
    "Titulo": "Tempo necessário para serviços (em horas)",
    "Conteudo": "Corte Masculino-> Leva 01 hora para ser feito / Corte Feminino-> Leva 02 horas para ser feito"
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]


In [ ]:
#Criando um Data Frame e organizando os dados em uma tabela
df = pd.DataFrame(documents)

In [ ]:
#definindo o modelo que vai gerar as embeddings
model = "models/embedding-001"

In [ ]:
#criando uma função que gera embeddings
def embed_fn(title, text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [ ]:
#Fazendo a função que gera embeddings percorrer por todo Data Frame
df["Embeddings"] = df.apply(lambda row: embed_fn(row["Titulo"],row["Conteudo"]), axis =1)

In [ ]:
def gerar_e_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]


  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice= np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteudo"]

In [ ]:
#Realizando uma consulta, e salvando qual documento condiz mais com o meu prompt
consulta = "Só posso ir aí depois do almoço. Que hrs vc atende?"

trecho = gerar_e_buscar_consulta(consulta, df, model)
print(trecho)

O horário de atendimento do salão de beleza é das 09:00 às 12:00 e das 14:00 às 22:00


In [ ]:
#Gerando uma resposta mais Criativa baseando-se no documento que foi retornado
generation_config = {
    "temperature": 0.25,
    "candidate_count":1,
}

prompt = f"Reescreva, sem adicionar informações que não façam parte do texto: ({trecho})"
model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)
response = model_2.generate_content(prompt)
print(response.text)

Horário de atendimento:
- 09:00 às 12:00
- 14:00 às 22:00
